# Advent of Code 2019 in Dyalog APL

By Stefan Kruger

Solutions to [AoC 2019](https://adventofcode.com/2019) in [Dyalog APL](https://www.dyalog.com/). Lots of help and inspiration from the friendly inhabitants of [The APL Orchard](https://chat.stackexchange.com/rooms/52405/the-apl-orchard) and [/r/apljk](https://www.reddit.com/r/apljk/).

Take the red pill.

## Day 1

https://adventofcode.com/2019/day/1

In [323]:
⎕IO←0
DAY1 ← ⍎¨ ⊃ ⎕NGET 'data/input01.data' 1

In [324]:
Fuel ← {0⌈¯2+⌊⍵÷3}

In [325]:
+/ Fuel DAY1 ⍝ part 1

3328306

In [327]:
+/{⍵=0:0 ⋄ (Fuel ⍵) + ∇Fuel ⍵} ¨ DAY1 ⍝ part 2

4989588

## Day 2

https://adventofcode.com/2019/day/2

In [314]:
⎕IO←0
DAY2 ← ⊢⌿⍎¨⎕CSV'data/input02.data'

In [318]:
]dinput
Intcode←{                                           ⍝ Intcode interpreter, mk1
    ⍺←0                                             
    op p1 p2 p3 ← 4↑⍺↓⍵                             ⍝ Skip anything before and take 4 cells
    op ∊ 1 2: (⍺+4)∇((op-1)⌷⍵[p1](+,×)⍵[p2])@p3 ⊢ ⍵ ⍝ Addition and multiplication
    op=99: ⍵[0]                                     ⍝ Exit
}

In [322]:
DAY2[1 2] ← 12 2
Intcode DAY2                                        ⍝ part 1: 10566835

10566835

For part two we want to find the two numbers that make the program output 19690720. We recurse over a list of all combinatons of ⍳100 and ⍳100 until we hit 19690720.

In [320]:
Run←{19690720= Intcode ((⊃1↑⍵)@1 2) ⊢ ⍺: ⊃1↑⍵ ⋄ ⍺∇1↓⍵} 

In [321]:
(noun verb) ← DAY2 Run {,⍳⍵ ⍵} 100 
verb + 100 × noun                                   ⍝ part 2: 2347

2347

## Day 3

https://adventofcode.com/2019/day/3

In [276]:
⎕IO←0
DAY3 ← 1(↑,∘⍎↓)¨ ⎕CSV'data/input03.data' ⍝ Convert each string a letter and a number.
OFFSETS ← 4 2 ⍴ 0 1 0 ¯1 1 0 ¯1 0
DIRS ← 'U' 'D' 'R' 'L'

In [277]:
Pairs←{↓(2÷⍨≢⍵) 2⍴⍵}                     ⍝ Turn flat lists to lists of pairs: 1 2 3 4 -> (1 2)(3 4)

In [306]:
]dinput
Follow←{                                 ⍝ Follow path ⍵, accumulate list of coord pairs visited
    ⍺←0 0
    0=≢⍵: ⍺                              ⍝ Quit if ⍵ is empty, and return accumulator ⍺
    (dir steps)←⊃⍵                       ⍝ Dir of travel, and number of steps, e.g. 'R' 200
    seq←,⌿OFFSETS[DIRS⍳dir;]∘.×1+⍳steps  ⍝ Expand out the steps into a vector
    origin←(≢seq) ⍴ ⊂¯2↑∊⍺               ⍝ Vector as long as seq; elem is the last pos we got to 
    ⍺,∊(seq+origin)∇1↓⍵                  ⍝ Sum the two vectors and append, then recurse on tail
}

In [311]:
(path1 path2)←Pairs∘Follow¨↓DAY3

For part one the task is to find the intersection point which has the lowest manhattan distance from the shared start point. 

In [312]:
crossings←1↓∪path1∩path2                 ⍝ Find intersections; drop the start point which is shared.
⌊/ (+/|) ¨ crossings                     ⍝ Part 1. Minimise Manhattan distance = sum of absolute values

860

Part two wants us instead to find the intersection point which has the lowest sum total steps along both paths.

In [313]:
⌊/+/(path1⍳crossings),⍪path2⍳crossings   ⍝ Part 2: intersect with smallest sum steps 

9238

## Day 4

https://adventofcode.com/2019/day/4

In [7]:
⎕IO←0
DAY4 ← 236491 {⍺+⍳⍵-⍺-1} 713787  ⍝ Inclusive range

In [8]:
]dinput
Valid←{
    digits←(6⍴10)⊤⍵           ⍝ Split into individual digits
    adj←¯1↓digits,⍪1⌽digits   ⍝ Matrix of adjacent digits
    (∧/≤/adj)∧∨/=/adj         ⍝ Each digit must be ≤ to its right, and least one pair of equal digits
}

In [9]:
+/Valid¨DAY4                  ⍝ part 1

1169

For part 2, the pairs rule is changed to mean that a pair cannot be part of a triple, or quad, or larger group of digits.

In [10]:
]dinput
Valid2←{
    runs←{¯2-/¯1,⍸1,⍨2≠/⍵}    ⍝ Count runs of equal digits 
    digits←(6⍴10)⊤⍵           ⍝ Split into individual digits
    adj←¯1↓digits,⍪1⌽digits   ⍝ Matrix of adjacent digits
    (∧/≤/adj)∧(2∊runs digits) ⍝ Same as before, but at least one pair not part of larger group
}

In [11]:
+/Valid2¨DAY4                        ⍝ part 2

757

## Day 5

More intcode stuff. The mode settings, jumps and comparison operations https://adventofcode.com/2019/day/5

In [335]:
⎕IO←0
DAY5 ← ⊢⌿⍎¨⎕CSV'data/input05.data'

In [336]:
]dinput
Intcode←{                                  ⍝ Intcode interpreter, mk2
    ⍺←0
    state ← ⍵
    eval ← {⍺=0: state[⍵] ⋄ ⍵}             ⍝ Position or immediate mode
    op p1 p2 p3 ← 4↑⍺↓⍵                    ⍝ Skip anything before and take 4 cells
    params ← 4 4 2 2 3 3 4 4 1             ⍝ Number of parameters by opcode
    ops ← (1+⍳8), 99                       ⍝ Valid opcodes
    m3 m2 m1 o2 o1 ← (5⍴10) ⊤ op           ⍝ Unpack the param modes
    op ← 10 ⊥ o2 o1                        ⍝ Repack the opcode, to go from (say) 1001 to 1
    count ← params[ops⍳op]                 ⍝ Number of params
    parmod ← m1 m2 m3 ,⍪ p1 p2 p3          ⍝ Table combining modes and params
    d1 d2 d3 ← 3↑eval/(¯1+count)↑parmod    ⍝ Pick relevant number of params, and apply modes
    ip ← ⍺+count                           ⍝ Advance ip by the width of current instr
    op ∊ 1 2: ip∇((op-1)⌷d1(+,×)d2)@p3 ⊢ ⍵ ⍝ Addition or multiplication
    op=3: ip∇input@p1⊢⍵                    ⍝ Input
    op=4: d1,ip∇⍵                          ⍝ Output
    op∊5 6: ⍵∇⍨ip d2⌷⍨(op-5)⌷d1(≠,=)0      ⍝ Jumps 
    op∊7 8: ip∇((op-7)⌷d1(<,=)d2)@p3 ⊢ ⍵   ⍝ Comparison < or =
    op=99: ⍬                               ⍝ Exit
}

In [337]:
input←1 ⋄ ¯1↑Intcode DAY5 ⍝ part 1

16209841

In [338]:
input ← 5 ⋄ Intcode DAY5 ⍝ part 2

8834787

## Day 6

https://adventofcode.com/2019/day/6

In [193]:
⍝ Read the file, and for each line, find all words (i.e split on paranthesis)
⍝ We then transpose the matrix and destructure into two separate vectors, 
⍝ holding parents and direct descendants respectively

(a b) ← ↓⍉↑ '\w+' ⎕S '&' ¨ ⊃⎕NGET'data/input06.data'1 ⍝ The '&' means return the matching string

In [196]:
parents ← b ⍳ a                  ⍝ parents gives the index into a of each item's parent.
path ← {3::⍵ ⋄ ⍵,∇⍵⊃parents}     ⍝ The 3::⍵ construct returns the arg in case of an INDEX ERROR
≢ ∊ path ¨ parents               ⍝ part 1 - apply path to each of the parent array, enlist and count

292387

For part two, find the path from YOU to SAN. As this is a tree, we can solve this as the paths of YOU to the centre plus the path of SAN to the centre with all common elements removed.

In [273]:
{¯2+≢⍺(∪~∩)⍵}/path¨b⍳'YOU' 'SAN' ⍝ Tally of union minus intersection, minus two (as counting edges, not nodes)

433

## Day 7

https://adventofcode.com/2019/day/7

Yet more intcoding for fun and profit. Previously we had the program be the right-hand argument of the Intcode function and the instruction pointer be the left-hand argument. For this task we need the Intcode interpreter be resumable, so the whole state must be external to the function so we can run several interconnected Intcode interpreters.


In [2]:
⎕IO←0
DAY7 ← ⊢⌿⍎¨⎕CSV'data/input07.data'

In [4]:
amps ← 5 5 ⍴ DAY7 0 ⍬ ⍬ 0 ⍝ Five intcode machine states

┌─────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────┬─┬┬┬─┐
│3 8 1001 8 10 8 105 1 0 0 21 42 67 84 109 122 203 284 365 446 99999 3 9 1002 9 3 9 1001 9 5 9 102 4 9 9 1001 9 3 9 4 9 99 3 9 1001 9 5 9 1002 9 3 9 1001 9 4 9 102 3 9 9 101 3 9 9 4 9 99 3 9 101 5 9 9 1002 9 3 9 101 5 9 9 4 9 99 3 9 102 5 9 9 101 5 9 9 102 3 9 9 101 3 9 9 102 2 9 9 4 9 99 3 9 101 2 9 9 1002 9 3 9 4 9 99 3 9 101 2 9 9 4 9 3 9 101 1 9 9 4 9 3 9 101 1 9 9 4 9 3 9 1001 9 1 9 4 9 3 9 101 1 9 9 4 9 3 9 1002 9 2 9 4 9 3 9 1002 9 2 9 4 9 3 9 1001 9 2 9 4 9 3 9 101 1 9 9 4 9 3 9 1002 9 2 9 4 9 99 3 9 1001 9 1 9 4 9 3 9 101 2 9 9 4 9 3 9 102 2 9 9 4 9 3 9 101 1 9 9 4 9 3 9 102 2 9 9 4 9 3 9 1001 9 1 9 4 9 3 9 101 1 9 9 4 9 3 9 1002 9 2 9 4 9 3 9 101 2 9 9 4 9 3 9 1002 9 2 9 4 9 99 3 9 101 2 9 9 4 9 3 9 101 2 9 9 4 9 3 9 101 2 9 9 4 9 3 9 101 1 9 9 4 9 3 9 101 1 9 9 4 9 3 9 102 2 9 9 4 9 3 9 1002 9 2 9 4 9 3 9 1002 9 2 9 4 9 3 9 101 2 9 9 4 9 3 9 1001 9 1 9 4 9 99 3 9 1001 9 1 9 4 9 3 9 101 1 9 9 4 9 3 9 102 2 9 9 4 9 3 9 1002 9 2 9 4 9 3 9 1001 9 2 9 4 9 3 9 1001 9 1 9 4 9 3 9 1001 9 2 9 4 9 3 9 1002 9 2 9 4 9 3 9 1002 9 2 9 4 9 3 9 102 2 9 9 4 9 99 3 9 102 2 9 9 4 9 3 9 1002 9 2 9 4 9 3 9 101 2 9 9 4 9 3 9 101 2 9 9 4 9 3 9 101 1 9 9 4 9 3 9 1002 9 2 9 4 9 3 9 101 1 9 9 4 9 3 9 1001 9 2 9 4 9 3 9 102 2 9 9 4 9 3 9 101 1 9 9 4 9 99│0│││0│
├─────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────┼─┼┼┼─┤
│3 8 1001 8 10 8 105 1 0 0 21 42 67 84 109 122 203 284 365 446 99999 3 9 1002 9 3 9 1001 9 5 9 102 4 9 9 1001 9 3 9 4 9 99 3 9 1001 9 5 9 1002 9 3 9 1001 9 4 9 102 3 9 9 101 3 9 9 4 9 99 3 9 101 5 9 9 1

In [ ]:
]dinput
Intcode←{                                          ⍝ Intcode interpreter, mk3 
    index ← ⍵
    size ← ≢amps
    state ip input output terminated ← ⍵⌷amps
    eval ← {⍺=0: state[0;⍵] ⋄ ⍵}                   ⍝ Position or immediate mode
    op p1 p2 p3 ← 4↑ip↓state                       ⍝ Skip anything before and take 4 cells
    params ← 4 4 2 2 3 3 4 4 1                     ⍝ Number of parameters by opcode
    ops ← (1+⍳8), 99                               ⍝ Valid opcodes
    m3 m2 m1 o2 o1 ← (5⍴10) ⊤ op                   ⍝ Unpack the param modes
    op ← 10 ⊥ o2 o1                                ⍝ Repack opcode, to go from (say) 1001 to 1
    count ← params[ops⍳op]                         ⍝ Number of params
    parmod ← m1 m2 m3 ,⍪ p1 p2 p3                  ⍝ Table combining modes and params
    d1 d2 d3 ← 3↑eval/(¯1+count)↑parmod            ⍝ Pick relevant number of params, and apply modes
    (0=≢input)∧op=3: ⍬                             ⍝ Input underrun; bail
    machines[⍵;1] +← count                         ⍝ Advance ip by the width of current instr
    op ∊ 1 2: ∇amps[⍵;0;p3]←((op-1)⌷d1(+,×)d2) ⊢ ⍵ ⍝ Addition or multiplication
    op=3: ∇amps[⍵;0;p1]←input ⊢ ⍵                  ⍝ Input
    op=4: ∇amps[size|⍵+1;2],←d1 ⊢ ⍵                ⍝ Output goes to the input buffer of next machine
    op∊5 6: ∇amps[⍵;1]←d2⌷⍨(op-5)⌷d1(≠,=)0 ⊢ ⍵     ⍝ Jumps
    op∊7 8: ∇amps[⍵;0;p3]←((op-7)⌷d1(<,=)d2) ⊢ ⍵   ⍝ Comparison < or =
    op=99: amps[⍵;4]←1 ⊢ ⍬                         ⍝ Exit
}